In [2]:
NUM_DOCS = 1000
NUM_THREADS = 4
BATCH_SIZE = NUM_DOCS/NUM_THREADS

In [3]:
import sys
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models import LdaModel

### Load Data

In [4]:
sys.executable

'C:\\Users\\Al\\Anaconda3\\envs\\tensorflow\\python.exe'

In [5]:
api.info("20-newsgroups")

{'checksum': 'c92fd4f6640a86d5ba89eaad818a9891',
 'description': 'The notorious collection of approximately 20,000 newsgroup posts, partitioned (nearly) evenly across 20 different newsgroups.',
 'fields': {'data': '',
  'id': 'original id inferred from folder name',
  'set': "marker of original split (possible values 'train' and 'test')",
  'topic': 'name of topic (20 variant of possible values)'},
 'file_name': '20-newsgroups.gz',
 'file_size': 14483581,
 'license': 'not found',
 'num_records': 18846,
 'parts': 1,
 'read_more': ['http://qwone.com/~jason/20Newsgroups/'],
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/20-newsgroups/__init__.py',
 'record_format': 'dict'}

In [11]:
%%time
import spacy

def addStopWords(stopWords, nlp):
    for stopWord in stopWords:
        lexeme = nlp.vocab[stopWord]
        lexeme.is_stop = True
        
def filterWords(word):
    return word.is_alpha and not word.is_stop

def convertWords(word):
    return word.lemma_.lower()

def cleanDoc(doc):
    return list(map(convertWords, filter(filterWords, doc)))

nlp = spacy.load("en", disable=['tagger', 'parser', 'ner', 'textcat'])
# Currently adding no other stop words
addStopWords([], nlp) 


nlpCorpus = []
for doc in nlp.pipe(corpus[:NUM_DOCS], n_threads=NUM_THREADS, batch_size=BATCH_SIZE):
    nlpCorpus.append(doc)
    
cleanCorpus = list(map(lambda doc: cleanDoc(doc), nlpCorpus))

CPU times: user 8.56 s, sys: 54.4 ms, total: 8.62 s
Wall time: 8.63 s


In [19]:
print(cleanCorpus[0])
print(cleanCorpus[0][0])
# [print(token.text) for token in cleanCorpus[0]]

['from', 'andrew', 'byler', 'subject', 're', 'serbian', 'genocide', 'work', 'god', 'organization', 'freshman', 'civil', 'engineering', 'carnegie', 'mellon', 'pittsburgh', 'pa', 'lines', 'vera', 'shanti', 'noyes', 'write', 'indicate', 'believe', 'predestination', 'correct', 'believe', 'predestination', 'believe', 'choose', 'accept', 'god', 'gift', 'salvation', 'fundamental', 'difference', 'resolve', 'of', 'course', 'i', 'believe', 'predestination', '-pron-', 'biblical', 'doctrine', 'roman', 'show', 'passage', 'furthermore', 'church', 'teach', 'predestination', 'begin', 'but', 'i', 'believe', 'predestination', 'mean', 'i', 'believe', 'free', 'men', 'freely', 'choose', 'course', 'life', 'affect', 'grace', 'god', 'however', 'unlike', 'calvinists', 'jansenists', 'i', 'hold', 'grace', 'resistable', 'end', 'idiocy', 'deny', 'universal', 'save', 'god', 'timothy', 'for', 'god', 'grace', 'save', 'but', 'elect', 'foreknow', 'predestine', 'receive', 'grace', 'final', 'perserverance', 'guarantee', 

### Add bigrams

In [6]:
# Need to get the generator from the dataset to use with gensim
generator = dataset.__iter__()
print(generator.__next__()) # example doc

{'topic': 'soc.religion.christian', 'data': 'From: db7n+@andrew.cmu.edu (D. Andrew Byler)\nSubject: Re: Serbian genocide Work of God?\nOrganization: Freshman, Civil Engineering, Carnegie Mellon, Pittsburgh, PA\nLines: 61\n\nVera Shanti Noyes writes;\n\n>this is what indicates to me that you may believe in predestination.\n>am i correct?  i do not believe in predestination -- i believe we all\n>choose whether or not we will accept God\'s gift of salvation to us.\n>again, fundamental difference which can\'t really be resolved.\n\nOf course I believe in Predestination.  It\'s a very biblical doctrine as\nRomans 8.28-30 shows (among other passages).  Furthermore, the Church\nhas always taught predestination, from the very beginning.  But to say\nthat I believe in Predestination does not mean I do not believe in free\nwill.  Men freely choose the course of their life, which is also\naffected by the grace of God.  However, unlike the Calvinists and\nJansenists, I hold that grace is resistabl

In [7]:
%%time
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

dictionary = Dictionary(cleanBigramCorpus)
bow = [dictionary.doc2bow(doc) for doc in cleanBigramCorpus]

tfidfModel = TfidfModel(bow)  # fit model
tfidf = [tfidfModel[doc] for doc in bow]

doc2Vec = [dictionary.doc2bow(doc) for doc in cleanBigramCorpus]

[ 536870912 1073741824 1610612735 2147483647]


## Pre-processing

In [8]:
import json
import gzip

corpusJson = []
file = gzip.open("/Users/jaybooth/gensim-data/20-newsgroups/20-newsgroups.gz")
corpusJson = [json.loads(line) for line in file]
file.close()
print(len(corpusJson))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jaybooth/gensim-data/20-newsgroups/20-newsgroups.gz'

In [ ]:
print(corpusJson[0])

In [ ]:
corpus = list(map(lambda jsonDoc: jsonDoc["data"],corpusJson))
corpusTopics = list(map(lambda jsonDoc: jsonDoc["topic"],corpusJson))

In [ ]:
import spacy
nlp = spacy.load("en")

In [ ]:
my_stop_words = []
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [ ]:
nlpCorpus = list(map(lambda doc: nlp(doc), corpus[:10]))

In [ ]:
def filterWords(word):
    return word.is_alpha and not word.is_stop

def convertWords(word):
    return word.lemma_.lower()

def cleanDoc(doc):
    return list(map(convertWords, filter(filterWords, doc)))

cleanCorpus = list(map(lambda doc: cleanDoc(doc), nlpCorpus))

In [ ]:
print(cleanCorpus[0])

In [ ]:
# We need to mess around with these
threshold = 1
minCount = 1
bigram = Phrases(cleanCorpus, min_count=minCount, threshold=threshold)

In [ ]:
cleanBigramCorpus = [bigram[doc] for doc in cleanCorpus]

In [ ]:
cleanBigramCorpus[0][:10]

In [ ]:
dictionary = Dictionary(cleanBigramCorpus)
bow = [dictionary.doc2bow(doc) for doc in cleanBigramCorpus]

In [ ]:
bow[0][:10]

In [ ]:
model = TfidfModel(bow)  # fit model
vector = model[bow[0]]

In [ ]:
print(vector[:20])

iterator = iter(dictionary.token2id.items())
for i in range(20):
    print(next(iterator))

In [ ]:
ldamodel = LdaModel(bow, num_topics=10, id2word=dictionary)